In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import jieba
import re

In [2]:
with open('vietnamese-stopwords.txt','r',encoding='utf-8') as file:
    stop_words = file.read()
    
stop_words = stop_words.split('\n')

In [3]:
data = pd.read_csv('ProductRaw.csv')
print(data.shape)
data.head(5)

(4404, 10)


,item_id,name,description,rating,price,list_price,brand,group,url,image
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,4.0,77000,300000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.5,132000,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nĐộ p...,4.8,299000,399000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://chuot-khong-day-logitech-m331-silent-p...,https://salt.tikicdn.com/cache/280x280/media/c...
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,THÔNG TIN CHI TIẾT\nThương hiệu Acome\nXuất xứ...,4.7,149000,350000,Acome,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://loa-bluetooth-5-0-kiem-dong-ho-bao-thu...,https://salt.tikicdn.com/cache/280x280/ts/prod...
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,THÔNG TIN CHI TIẾT\nThương hiệu Apple\nXuất xứ...,4.8,5090000,8500000,Apple,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-apple-airpods-pro-t...,https://salt.tikicdn.com/cache/280x280/ts/prod...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      4404 non-null   int64  
 1   name         4404 non-null   object 
 2   description  4401 non-null   object 
 3   rating       4404 non-null   float64
 4   price        4404 non-null   int64  
 5   list_price   4404 non-null   int64  
 6   brand        4404 non-null   object 
 7   group        4404 non-null   object 
 8   url          4404 non-null   object 
 9   image        4404 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 344.2+ KB


In [5]:
data.columns

Index(['item_id', 'name', 'description', 'rating', 'price', 'list_price',
       'brand', 'group', 'url', 'image'],
      dtype='object')

In [6]:
data.drop(columns=['url','image'],inplace=True)

In [7]:
data.drop_duplicates(inplace=True)
data.shape

(4373, 8)

In [8]:
data.isna().sum()

item_id        0
name           0
description    3
rating         0
price          0
list_price     0
brand          0
group          0
dtype: int64

In [9]:
data.dropna(inplace=True)

In [10]:
data.columns

Index(['item_id', 'name', 'description', 'rating', 'price', 'list_price',
       'brand', 'group'],
      dtype='object')

In [11]:
for col in list(data.columns):
    print('%s: %d' %(col,len(data[col].unique())))
    print()

item_id: 4370

name: 4327

description: 4370

rating: 33

price: 1782

list_price: 954

brand: 521

group: 248



In [12]:
data.describe().round(2).drop('item_id',axis=1)

,rating,price,list_price
count,4370.00,4370.00,4370.00
mean,3.67,2764025.27,3894278.07
std,1.80,5545780.84,7903281.13
min,0.00,7000.00,12000.00
25%,3.80,150000.00,279000.00
50%,4.50,486500.50,790000.00
75%,4.80,2679000.00,3590000.00
max,5.00,62690000.00,82990000.00


In [13]:
# same name, different id, description
test_df = pd.DataFrame(data['name'].value_counts())
test_df[test_df['name']>1]

,name
Máy sấy Electrolux 7 Kg EDV705HQWA,3
"Bộ giá đỡ điện thoại, máy ảnh 3 chân tripod",3
Máy sấy Electrolux 8 Kg EDV805JQWA,3
Máy Nước Nóng Ariston SB35E-VN (3500W),3
Máy Nước Nóng Ariston SM45PE-VN (4500W),2
Máy sấy Electrolux 8 kg EDS805KQSA,2
Điện thoại Masstel Fami 12 - Hàng Chính Hãng,2
Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,2
USB Thu Wifi 5G Tốc Độ 1300Mbps - Băng tần kép,2
Tủ Lạnh Sharp Inverter 556L SJ-FX631V-SL,2


In [14]:
moreid = list(test_df[test_df['name']>1].index)
len(moreid)

39

In [15]:
data[data['name']==moreid[0]]

,item_id,name,description,rating,price,list_price,brand,group
3414,29245829,Máy sấy Electrolux 7 Kg EDV705HQWA,THÔNG TIN CHI TIẾT\nBảng điều khiển Điện tử\nT...,4.2,6990000,9990000,Electrolux,Điện Tử - Điện Lạnh/Máy sấy quần áo
3459,29247864,Máy sấy Electrolux 7 Kg EDV705HQWA,THÔNG TIN CHI TIẾT\nBảng điều khiển Điều khiển...,4.6,7079000,9990000,Electrolux,Điện Tử - Điện Lạnh/Máy sấy quần áo
3792,29244640,Máy sấy Electrolux 7 Kg EDV705HQWA,THÔNG TIN CHI TIẾT\nBảng điều khiển Điện tử\nT...,4.0,6949000,9990000,Electrolux,Điện Tử - Điện Lạnh/Máy sấy quần áo


In [16]:
print(len(data.brand.unique()))
data['brand'].replace('\tOEM','OEM',inplace=True)
data['brand'].replace(' EZVIZ','Ezviz',inplace=True)
len(data.brand.unique())

521


519

In [17]:
import pandas
from sklearn.preprocessing import LabelEncoder

def label_en(df,col_name):
    encoder = LabelEncoder()
    df['label_%s' %col_name] = encoder.fit_transform(df[col_name])
    new_name = 'label_%s' %col_name
    print('new col:',new_name)
    return df

data = label_en(data,'group')

new col: label_group


In [18]:
data['name'][0]

'Tai nghe Bluetooth Inpods 12 - Cảm biến vân tay, chống nước,màu sắc đa dạng- 5 màu sắc lựa chọn'

In [19]:
import string
exclist = string.punctuation + string.digits
table_ = str.maketrans('', '', exclist)

data['new'] = data['name'] +' ' + data['description'].apply(lambda x: x.replace('\n',' '))
data['new'] = data['new'].apply(lambda x: x.translate(table_))
data['new'] = data['new'].apply(lambda x: re.sub(r'\W+|[0-9]', ' ', x.lower()))

ending_txt = 'giá sản phẩm trên tiki đã bao gồm thuế theo luật hiện hành tuy nhiên tuỳ vào từng loại sản phẩm hoặc phương thức địa chỉ giao hàng mà có thể phát sinh thêm chi phí khác như phí vận chuyển phụ phí hàng cồng kềnh'
thong_tin = 'thông tin chi tiết'
mo_ta = 'mô tả sản phẩm'

data['new'] = data['new'].apply(lambda x: x.replace(ending_txt, ' '))
data['new'] = data['new'].apply(lambda x: x.replace(thong_tin, ' '))
data['new'] = data['new'].apply(lambda x: x.replace(mo_ta, ' '))

data['new'] = data['new'].apply(lambda x: re.sub(r'\s+', ' ', x))


data['new']= data['new']\
             + 'brand_'+ data['brand'].apply(lambda x: x.lower()) + ' ' \
             + 'group_'+ data['label_group'].apply(lambda x: str(x)) + ' ' \
             + 'price_'+ data['list_price'].apply(lambda x: str(x)) + ' ' \
             + 'rating_'+ data['rating'].apply(lambda x: str(int(x)))

data['new'][0]

'tai nghe bluetooth inpods cảm biến vân tay chống nướcmàu sắc đa dạng màu sắc lựa chọn dung lượng pin thời gian pin thời gian nghe nhạc liên tục từ h thời gian sạc đầy chỉ khoảng p thời gian chờ lên tới giờ bluetooth thương hiệu oem xuất xứ thương hiệu trung quốc độ nhạy cảm biến vân tay model i loại jack cắm usb cable trọng lượng g thời gian sử dụng thời gian nghe nhạc liên tục từ h sku inpod là phiên bản nâng cấp mới nhất tai nghe bluetooth có thiết kế tỉ lệ chuẩn với tai airpod chính hãng lược bỏ nút bấm trên thân tai thay vào đó là nút cảm ứng sử dụng dễ dàng và thuận tiện hơn thao tác nhận cuộc gọi đến chuyển nhạcmột cách dễ dàng chỉ bằng chạm bluetooth mới nhất cho kết nối vô cùng ổn định bluetooth mới nhất cho kết nối vô cùng ổn định tai nghe tự kết nối với nhau khi được lấy ra từ dock sạc chất âm và thời lượng pin được cải thiện tối ưu dock sạc tiện lợi cho tới lần sạc đầy pin tai nghe các thiết bị tương thích các thiết bị bluetooth như cho xiaomi samsung apple huawei oppo vivo

In [20]:
data['new'][1]

'tai nghe bluetooth không dây f true wireless dock sạc có led báo pin kép dung lượng pin mah thời gian pin thời gian nghe nhạc giờ thời gian đàm thoại giờ bluetooth v thương hiệu oem xuất xứ thương hiệu trung quốc độ nhạy cao model f loại jack cắm không dâybluetooth trọng lượng g tần số hz thời gian sử dụng thời gian nghe nhạc giờ thời gian đàm thoại giờ hỗ trợ đàm thoại có sku sử dụng chip bluetooth mạnh mẽ cho kết nối ổn định nhanh hơn với khoảng cách kết nối xa m và tiết kiệm năng lượng pin tự khỏi động khi nhấc ra khỏi hộp sạc tự động kết nối bluetooth với điện thoại khi ở trạng thai sẵn sàng tự ngắt kế nối khi bỏ vào hộp sạc khoảng cách hoạt động phạm vi m công nghệ khử tiếng ồn cho âm thanh trung thực nhất và chất lượng cao cvc dung lượng pin từng tai mah trọng lượng tai nghe g rất nhẹ và khít cho cảm giác thoải mái dung lương dock sạc mah có thể sạc cho tai nghe lần không chỉ sạc cho tai nghe mà còn có thể sử dụng để sạc cho điện thoại thời gian nghe nhạc giờ thời gian đàm thoại

In [21]:
data['new'][54]

'sim g viettel d trọn gói g năm không cần nạp tiền chính hãng mẫu ngẫu nhiên thương hiệu viettel xuất xứ thương hiệu việt nam xuất xứ việt nam hướng dẫn bảo quản vì sim không có điều kiện kích hoạt từng chiếc nên sim bán ra đã được kích hoạt trước và shop không quản lý được hàng hóa xuất ra từ kho tiki nên sim có thể mất ít ngày nếu sim mất quá ngày sử dụng shop xin phép đền bù số ngày sử dụng bị mất vượt quá cam kết theo công thức số tiền mua sim ngày số ngày vượt quá sim tháng nhưng trên app my viettel sẽ báo thời gian từng tháng tức thời gian sử dụng gbtháng nó không phải thời gian còn lại của sim thời gian kiểm tra trên cũng không phải là thời gian khuyến mãi còn lại của sim để kiểm tra chính xác quý khách có thể nhìn trên bìa sim nhà bán đã ghi chú sẵn hoặc gọi hỏi ngày đăng ký gói cước d để biết chính xác hơn lưu ý ngày kích hoạt sim khác với ngày kích hoạt gói cước d nên cần phân biệt rõ khi hỏi số hỗ trợ xử lý zalo viber đã gán sẵn trên sim các trường hợp hư hỏng hay không sử d

In [22]:
data['new_wt'] = data['new'].apply(lambda x: word_tokenize(x,format='text'))
data['new_wt'][1]

'tai nghe bluetooth không dây f true wireless dock sạc có led báo pin kép dung_lượng pin mah thời_gian pin thời_gian nghe nhạc giờ thời_gian đàm_thoại giờ bluetooth v thương_hiệu oem xuất_xứ thương_hiệu trung_quốc độ_nhạy cao model f loại jack cắm không dâybluetooth trọng_lượng g tần_số hz thời_gian sử_dụng thời_gian nghe nhạc giờ thời_gian đàm_thoại giờ hỗ_trợ đàm_thoại có sku sử_dụng chip bluetooth mạnh_mẽ cho kết_nối ổn_định nhanh hơn với khoảng_cách kết_nối xa m và tiết_kiệm năng_lượng pin tự khỏi động khi nhấc ra khỏi hộp sạc tự_động kết_nối bluetooth với điện_thoại khi ở trạng thai sẵn_sàng tự ngắt kế nối khi bỏ vào hộp sạc khoảng_cách hoạt_động phạm_vi m công_nghệ khử tiếng ồn cho âm_thanh trung_thực nhất và chất_lượng cao cvc dung_lượng pin từng tai mah trọng_lượng tai_nghe g rất nhẹ và khít cho cảm_giác thoải_mái dung lương dock sạc mah có_thể sạc cho tai_nghe lần không chỉ sạc cho tai_nghe mà_còn có_thể sử_dụng để sạc cho điện_thoại thời_gian nghe nhạc giờ thời_gian đàm_thoại

# Gensim

In [23]:
products_gem = [[text for text in x.split()] for x in data['new_wt']]
len(products_gem)

4370

In [24]:
products_gem_re = [[t for t in text if not t in stop_words] for text in products_gem]
products_gem_re[1]

['tai',
 'bluetooth',
 'không',
 'dây',
 'f',
 'true',
 'wireless',
 'dock',
 'sạc',
 'led',
 'báo',
 'pin',
 'kép',
 'dung_lượng',
 'pin',
 'mah',
 'pin',
 'nhạc',
 'đàm_thoại',
 'bluetooth',
 'v',
 'thương_hiệu',
 'oem',
 'xuất_xứ',
 'thương_hiệu',
 'trung_quốc',
 'độ_nhạy',
 'model',
 'f',
 'jack',
 'cắm',
 'không',
 'dâybluetooth',
 'trọng_lượng',
 'g',
 'tần_số',
 'hz',
 'nhạc',
 'đàm_thoại',
 'đàm_thoại',
 'sku',
 'chip',
 'bluetooth',
 'mạnh_mẽ',
 'kết_nối',
 'ổn_định',
 'kết_nối',
 'm',
 'tiết_kiệm',
 'năng_lượng',
 'pin',
 'động',
 'nhấc',
 'hộp',
 'sạc',
 'tự_động',
 'kết_nối',
 'bluetooth',
 'điện_thoại',
 'trạng',
 'thai',
 'sẵn_sàng',
 'ngắt',
 'kế',
 'nối',
 'hộp',
 'sạc',
 'hoạt_động',
 'phạm_vi',
 'm',
 'công_nghệ',
 'khử',
 'tiếng',
 'ồn',
 'âm_thanh',
 'trung_thực',
 'chất_lượng',
 'cvc',
 'dung_lượng',
 'pin',
 'tai',
 'mah',
 'trọng_lượng',
 'tai_nghe',
 'g',
 'nhẹ',
 'khít',
 'cảm_giác',
 'thoải_mái',
 'dung',
 'lương',
 'dock',
 'sạc',
 'mah',
 'sạc',
 'tai_nghe',

In [25]:
dictionary = corpora.Dictionary(products_gem_re)
feature_cnt = len(dictionary.token2id)
feature_cnt

22489

In [26]:
corpus = [dictionary.doc2bow(text) for text in products_gem_re]

In [27]:
tfidf = models.TfidfModel(corpus)
index = similarities.SparseMatrixSimilarity(tfidf[corpus],num_features=feature_cnt)

In [28]:
data.columns

Index(['item_id', 'name', 'description', 'rating', 'price', 'list_price',
       'brand', 'group', 'label_group', 'new', 'new_wt'],
      dtype='object')

In [29]:
def product_rec_gs(product_id, rec_num):
    product_idx = data[data['item_id']==int(product_id)].index[0]
    print(product_idx)
    
    df = data[data['item_id']==int(product_id)]
    product = df['new_wt'].to_string(index=False).split()
    kw_vector = dictionary.doc2bow(product)
    sim = index[tfidf[kw_vector]]
    
    gs_dict = {}
    idx_lst = [product_idx]

    for i in range(len(sim)):
        a = sim[i]
        if (i!= product_idx)&(a>=0.35):
            gs_dict[i] = a

    idx_lst = idx_lst + sorted(gs_dict, key=gs_dict.get,reverse=True)
    df_new = data[['item_id','name', 'description', 'rating', 'list_price', 'brand', 
                   'group']].iloc[idx_lst]
    
    return df_new.head(rec_num + 1), gs_dict

In [30]:
gs_rec, gs_dict = product_rec_gs(70772235, 5)
gs_rec

77


,item_id,name,description,rating,list_price,brand,group
77,70772235,Tai nghe bluetooth nhét tai không dây tai phon...,THÔNG TIN CHI TIẾT\nDung lượng pin 300mAh\nBlu...,4.5,599000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
155,74141526,Tai nghe có dây nhét tai màu trắng - Hàng Chín...,THÔNG TIN CHI TIẾT\nThương hiệu PKCB\nXuất xứ ...,4.7,200000,PKCB,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
500,74623725,Tai Nghe Nhét Tai 3.5mm Dành Cho IOS Android T...,THÔNG TIN CHI TIẾT\nThương hiệu PKCB\nXuất xứ ...,4.7,200000,PKCB,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
216,54406343,Tai Nghe Bluetooth Nhét Tai Không Dây 5.0 Dalu...,THÔNG TIN CHI TIẾT\nThương hiệu Dalugi\nXuất x...,4.0,950000,Dalugi,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
162,14790642,Tai nghe Bluetooth nhét tai không dây SS315,THÔNG TIN CHI TIẾT\nThương hiệu OEM\nLưu ý Hỗ ...,4.8,150000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
299,74247140,Tai nghe không dây Bluetooth Inpod 12 Simple -...,THÔNG TIN CHI TIẾT\nBluetooth 5\nThương hiệu O...,4.7,249000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...


In [31]:
gs_dict

{13: 0.35102713,
 24: 0.350873,
 45: 0.361041,
 55: 0.39123234,
 147: 0.3743582,
 155: 0.5751958,
 162: 0.4642386,
 216: 0.47678986,
 219: 0.42904115,
 244: 0.42223436,
 279: 0.38716593,
 299: 0.43356797,
 410: 0.40183482,
 500: 0.533889,
 581: 0.37641206,
 593: 0.39093444,
 716: 0.37464035,
 844: 0.3512574,
 914: 0.35634553}

# Cosin Similarity

In [32]:
from numpy import dot
from numpy.linalg import norm

In [33]:
tf = TfidfVectorizer(analyzer='word',min_df=0,stop_words=stop_words)
tfidf_matrix = tf.fit_transform(data['new_wt'])

In [34]:
cosine_similarities = cosine_similarity(tfidf_matrix,tfidf_matrix)
cosine_similarities

array([[1.        , 0.46774021, 0.0822364 , ..., 0.00845639, 0.00577978,
        0.05176316],
       [0.46774021, 1.        , 0.08106923, ..., 0.01567303, 0.02917938,
        0.08440691],
       [0.0822364 , 0.08106923, 1.        , ..., 0.03114121, 0.0182702 ,
        0.06779557],
       ...,
       [0.00845639, 0.01567303, 0.03114121, ..., 1.        , 0.00433548,
        0.02701191],
       [0.00577978, 0.02917938, 0.0182702 , ..., 0.00433548, 1.        ,
        0.04144776],
       [0.05176316, 0.08440691, 0.06779557, ..., 0.02701191, 0.04144776,
        1.        ]])

In [35]:
cs_df = pd.DataFrame(cosine_similarities,columns=list(data['item_id']))
cs_df

,48102821,52333193,299461,57440329,38458616,12567795,4897521,47321729,2638665,25078249,...,19407194,53716886,22967676,29245831,68495317,14497425,52294251,29246309,48574854,60228865
0,1.000000,0.467740,0.082236,0.187751,0.199499,0.262998,0.200143,0.135431,0.156540,0.048670,...,0.176739,0.021831,0.011263,0.011005,0.050846,0.013545,0.049832,0.008456,0.005780,0.051763
1,0.467740,1.000000,0.081069,0.200798,0.336250,0.353233,0.286062,0.171699,0.240811,0.071072,...,0.166626,0.016180,0.020253,0.012571,0.058586,0.035544,0.068327,0.015673,0.029179,0.084407
2,0.082236,0.081069,1.000000,0.058437,0.067493,0.039736,0.056110,0.043132,0.035903,0.408286,...,0.090947,0.043767,0.010192,0.033116,0.014417,0.045180,0.067562,0.031141,0.018270,0.067796
3,0.187751,0.200798,0.058437,1.000000,0.074367,0.061713,0.030823,0.111426,0.037081,0.034079,...,0.172040,0.015576,0.044896,0.023997,0.116824,0.018498,0.047064,0.019785,0.031232,0.084522
4,0.199499,0.336250,0.067493,0.074367,1.000000,0.641667,0.241983,0.040371,0.214447,0.065770,...,0.085479,0.027074,0.036360,0.019587,0.072031,0.049951,0.077199,0.019873,0.006385,0.106382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4365,0.013545,0.035544,0.045180,0.018498,0.049951,0.027566,0.046568,0.023329,0.031969,0.041071,...,0.043906,0.106206,0.017186,0.061162,0.046161,1.000000,0.027835,0.051799,0.008345,0.027259
4366,0.049832,0.068327,0.067562,0.047064,0.077199,0.038493,0.088160,0.020308,0.076011,0.041161,...,0.120317,0.021329,0.043457,0.010462,0.123429,0.027835,1.000000,0.013152,0.016049,0.284660
4367,0.008456,0.015673,0.031141,0.019785,0.019873,0.012030,0.025084,0.009678,0.013413,0.021167,...,0.012232,0.168110,0.006137,0.907414,0.006192,0.051799,0.013152,1.000000,0.004335,0.027012
4368,0.005780,0.029179,0.018270,0.031232,0.006385,0.007533,0.034724,0.032552,0.014139,0.022037,...,0.034142,0.003168,0.038103,0.005994,0.011534,0.008345,0.016049,0.004335,1.000000,0.041448


In [36]:
def product_rec_cs(product_id, rec_num):
    cs_similar = cs_df[[product_id]].sort_values(product_id,ascending=False)
    idx_lst = list(cs_similar.index)
    df_new = data[['item_id','name', 'description', 'rating', 'list_price', 'brand', 
                   'group']].iloc[idx_lst]
    
    return df_new.head(rec_num + 1), cs_similar

In [37]:
cs_rec, cs_similar = product_rec_cs(70772235, 5)
cs_rec

,item_id,name,description,rating,list_price,brand,group
77,70772235,Tai nghe bluetooth nhét tai không dây tai phon...,THÔNG TIN CHI TIẾT\nDung lượng pin 300mAh\nBlu...,4.5,599000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
147,22413470,Tai Nghe Bluetooth Air.podes Cảm Ứng Công Nghệ...,THÔNG TIN CHI TIẾT\nDung lượng pin 250\nThời g...,4.0,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
216,54406343,Tai Nghe Bluetooth Nhét Tai Không Dây 5.0 Dalu...,THÔNG TIN CHI TIẾT\nThương hiệu Dalugi\nXuất x...,4.0,950000,Dalugi,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.5,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
484,26093257,Tai nghe không dây bluetooth true wireless F9...,THÔNG TIN CHI TIẾT\nDung lượng pin 3500\nThời ...,4.7,380000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
411,72560404,Tai nghe chụp tai bluetooth chống ồn GS-H3 cao...,THÔNG TIN CHI TIẾT\nDung lượng pin 400\nThời g...,4.7,950000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...


In [38]:
cs_similar

,70772235
77,1.000000
147,0.578617
216,0.560460
1,0.526218
484,0.520810
...,...
2951,0.000961
1281,0.000640
2397,0.000629
2847,0.000603


# Đánh giá tổng thể mô hình Content-Based Filtering
- Tiền xử lý dữ liệu:
    - Sử dụng cột: name, desciption, brand, list_price, rating
    - Gắn nhãn cột brand
    - Cột rating chuyển sang kiểu int


- Cả 2 mô hình đều cho kết quả tốt
- Mô hình Cosin Similarity có nhiều đề suất hơn mô hình Gensim
- Mô hình Gensim đề suất sản phẩm có mức giá chênh lệch nhiều hơn mô hình Cosin Similarity

### => Áp dụng mô hình Cosin Similarity

### save files

In [39]:
cs_df

,48102821,52333193,299461,57440329,38458616,12567795,4897521,47321729,2638665,25078249,...,19407194,53716886,22967676,29245831,68495317,14497425,52294251,29246309,48574854,60228865
0,1.000000,0.467740,0.082236,0.187751,0.199499,0.262998,0.200143,0.135431,0.156540,0.048670,...,0.176739,0.021831,0.011263,0.011005,0.050846,0.013545,0.049832,0.008456,0.005780,0.051763
1,0.467740,1.000000,0.081069,0.200798,0.336250,0.353233,0.286062,0.171699,0.240811,0.071072,...,0.166626,0.016180,0.020253,0.012571,0.058586,0.035544,0.068327,0.015673,0.029179,0.084407
2,0.082236,0.081069,1.000000,0.058437,0.067493,0.039736,0.056110,0.043132,0.035903,0.408286,...,0.090947,0.043767,0.010192,0.033116,0.014417,0.045180,0.067562,0.031141,0.018270,0.067796
3,0.187751,0.200798,0.058437,1.000000,0.074367,0.061713,0.030823,0.111426,0.037081,0.034079,...,0.172040,0.015576,0.044896,0.023997,0.116824,0.018498,0.047064,0.019785,0.031232,0.084522
4,0.199499,0.336250,0.067493,0.074367,1.000000,0.641667,0.241983,0.040371,0.214447,0.065770,...,0.085479,0.027074,0.036360,0.019587,0.072031,0.049951,0.077199,0.019873,0.006385,0.106382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4365,0.013545,0.035544,0.045180,0.018498,0.049951,0.027566,0.046568,0.023329,0.031969,0.041071,...,0.043906,0.106206,0.017186,0.061162,0.046161,1.000000,0.027835,0.051799,0.008345,0.027259
4366,0.049832,0.068327,0.067562,0.047064,0.077199,0.038493,0.088160,0.020308,0.076011,0.041161,...,0.120317,0.021329,0.043457,0.010462,0.123429,0.027835,1.000000,0.013152,0.016049,0.284660
4367,0.008456,0.015673,0.031141,0.019785,0.019873,0.012030,0.025084,0.009678,0.013413,0.021167,...,0.012232,0.168110,0.006137,0.907414,0.006192,0.051799,0.013152,1.000000,0.004335,0.027012
4368,0.005780,0.029179,0.018270,0.031232,0.006385,0.007533,0.034724,0.032552,0.014139,0.022037,...,0.034142,0.003168,0.038103,0.005994,0.011534,0.008345,0.016049,0.004335,1.000000,0.041448


In [40]:
dictionary

In [41]:
data.to_csv('for_streamlit/nlp_data.csv',index=False)

In [42]:
import pickle

In [43]:
# data
with open('for_streamlit/nlp_data.pkl','wb') as file:
    pickle.dump(data,file)

In [44]:
# dictionary
with open('for_streamlit/gen_dict.pkl','wb') as file:
    pickle.dump(dictionary,file)

In [45]:
# tfidf
with open('for_streamlit/gen_tfidf.pkl','wb') as file:
    pickle.dump(tfidf,file)

In [46]:
# index
with open('for_streamlit/gen_index.pkl','wb') as file:
    pickle.dump(index,file)

In [47]:
# cs_df
with open('for_streamlit/cs_df.pkl','wb') as file:
    pickle.dump(cs_df,file)